In [22]:
// Cache model example 4 - Testing different replacement strategies
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.mva.*
import jline.solvers.nc.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
import jline.util.matrix.Matrix

// Model parameters
val n = 5  // number of items
val m = Matrix("[2,1]")  // cache capacity
val alpha = 1.0  // Zipf parameter

// Replacement strategies to test
val replStrat = arrayOf(ReplacementStrategy.RR, ReplacementStrategy.FIFO, ReplacementStrategy.LRU)

// Storage for results
val avgTable = mutableListOf<Any>()
val solver = mutableListOf<Any>()
val ctmcHitRatio = mutableListOf<Double>()
val mvaHitRatio = mutableListOf<Double>()
val ncHitRatio = mutableListOf<Double>()

println("repl-strategy, ctmc, mva, nc:")

In [ ]:
// Test each replacement strategy
for (strategy in replStrat) {
    // Create model
    val model = Network("model")
    val source = Source(model, "Source")
    val cacheNode = Cache(model, "Cache", n, m, strategy)
    val sink = Sink(model, "Sink")
    
    // Create classes
    val jobClass = OpenClass(model, "InitClass", 0)
    val hitClass = OpenClass(model, "HitClass", 0)
    val missClass = OpenClass(model, "MissClass", 0)
    
    // Set arrival
    source.setArrival(jobClass, Exp(2.0))
    
    // Zipf-like item references
    val pAccess = Zipf(alpha, n)
    cacheNode.setRead(jobClass, pAccess)
    
    // Configure cache classes
    cacheNode.setHitClass(jobClass, hitClass)
    cacheNode.setMissClass(jobClass, missClass)
    
    // Routing
    val P = model.initRoutingMatrix()
    P.set(jobClass, jobClass, source, cacheNode, 1.0)
    P.set(hitClass, hitClass, cacheNode, sink, 1.0)
    P.set(missClass, missClass, cacheNode, sink, 1.0)
    model.link(P)
    
    // Solver CTMC
    try {
        val ctmcSolver = CTMC(model, "keep", false, "cutoff", 1, "seed", 1, "verbose", false)
        solver.add(ctmcSolver)
        avgTable.add(ctmcSolver.avgNodeTable)
        val hr = cacheNode.hitRatio.value()
        ctmcHitRatio.add(hr)
    } catch (e: Exception) {
        ctmcHitRatio.add(Double.NaN)
    }
    
    // Solver MVA
    try {
        val mvaSolver = MVA(model, "seed", 1, "verbose", false)
        solver.add(mvaSolver)
        model.reset()
        avgTable.add(mvaSolver.avgNodeTable)
        val hr = cacheNode.hitRatio.value()
        mvaHitRatio.add(hr)
    } catch (e: Exception) {
        mvaHitRatio.add(Double.NaN)
    }

    // Solver NC
    try {
        val ncSolver = NC(model, "seed", 1, "verbose", false)
        if (ncSolver.supports(model)) {
            solver.add(ncSolver)
            model.reset()
            avgTable.add(ncSolver.avgNodeTable)
            val hr = cacheNode.hitRatio.value()
            ncHitRatio.add(hr)
        } else {
            ncHitRatio.add(Double.NaN)
        }
    } catch (e: Exception) {
        ncHitRatio.add(Double.NaN)
    }
    
    // Print results for this strategy
    val strategyName = when (strategy) {
        ReplacementStrategy.RR -> "rr"
        ReplacementStrategy.FIFO -> "fifo"
        ReplacementStrategy.LRU -> "lru"
        else -> strategy.toString().lowercase()
    }
    
    println(String.format("%s: %.8f, %.8f, %.8f", strategyName, ctmcHitRatio.last(), mvaHitRatio.last(), ncHitRatio.last()))
}